<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./04_aws_open_data_registry.ipynb"><< 04 - AWS - Open Data Registry</a><span style="float:right;"><a href="./11_covid19_case_study_s5p_anomaly_map.ipynb">11 - Covid-19 case study - Sentinel-5P anomaly map>></a></span>

# Google Earth Engine

### About

<img src="./img/gee_logo_2.png" align="right" width="35%"></img>
[Google Earth Engine (GEE)](https://earthengine.google.com/) is a planetary-scale platform for Earth science data & analysis. 

There are several ways to work with the Google Earth Engine:
* [Code Editor]( code.earthengine.google.com), a web-based IDE in Javascript
* [Client libraries](https://github.com/google/earthengine-api) provide Javascript and Python wrapper functions for the Earth Engine API

You have to [sign up](https://signup.earthengine.google.com/) for GEE.

### Data

Earth Engine's data archive includes:
* **Weather and Climate Data**
    * A selection of ERA5 reanalysis
      * [ECMWF/ERA5/DAILY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY)
      * [ECMWF/ERA5/MONTHLY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY)
    * TRMM precipitation
* **Imagery**
    * Landsat
    * Sentinels, e.g
      * [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
      * [Sentinel-5P](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
      * [Sentinel-3](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S3_OLCI)
    * MODIS
    
... and many more. Have a look yourself at the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/).

In case you are interested in uploading your own data in Google Earth Engine, see here a [workflow](https://github.com/jwagemann/era5_in_gee) how to upload ERA5 data, using [manifest uploads](https://developers.google.com/earth-engine/image_manifest). 

### How to retrieve data?

Required libraries:
* [Earth Engine Python API](https://developers.google.com/earth-engine/python_install)
* [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)

<hr>

#### Load required libraries

In [8]:
import ee

from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipyleaflet
import ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

In [7]:
# Initialize ee with access token
ee.Initialize()

Function below is taken from Tyler Erickson's [notebooks](https://github.com/tylere/EEUS2018-JupyterSession/blob/master/02%20-%20Interactive%20Maps.ipynb) for his Interactive Jupyter session at EEUS18.

In [18]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

#### Load an ERA5/MONTHLY image and get image information

`ee.Image`

In [11]:
img_test = ee.Image('ECMWF/ERA5/MONTHLY/200001')
img_test.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'minimum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'maximum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'dewpoint_2m_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'total_precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_trans

#### Select one specific parameter

In [16]:
t2m = img_test.select('mean_2m_air_temperature')
tp = img_test.select('total_precipitation')
print(tp.getInfo())

{'type': 'Image', 'bands': [{'id': 'total_precipitation', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1440, 721], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}], 'version': 1578581373737848, 'id': 'ECMWF/ERA5/MONTHLY/200001', 'properties': {'system:time_start': 946684800000, 'month': 1, 'year': 2000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]}, 'system:time_end': 949363200000, 'system:asset_size': 37702939, 'system:index': '200001'}}


#### Get image url for visualization

In [19]:
t2m_url = GetTileLayerUrl(t2m)
tp_url = GetTileLayerUrl(tp)

In [22]:
t2m_url

'https://earthengine.googleapis.com/map/projects/earthengine-legacy/maps/83d832a130d6629d256eb9b3ac11202e-b20f701514af99b1b18f832b1dee2418/{z}/{x}/{y}?token='

In [26]:
map1 = ipyleaflet.Map(
    zoom=2,
    layout={'height':'500px'},
)

map1.add_layer(ipyleaflet.TileLayer(url=t2m_url))
#map1.add_layer(ipyleaflet.TileLayer(url=tp_url))

# Adding some fance controls to the map, e.g. layers conrol, FullScreenControl
map1.add_control(ipyleaflet.LayersControl())
control = FullScreenControl()
map1.add_control(control)

map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

#### Get image url and apply visualization params to it

In [27]:
t2m_url = GetTileLayerUrl(t2m.visualize(min=250, max=310, palette=['#000080','#0000D9','#4000FF','#8000FF','#0080FF'\
                                                                   ,'#00FFFF','#00FF80','#80FF00','#DAFF00','#FFFF00','#FFF500','#FFDA00','#FFB000','#FFA400','#FF4F00','#FF2500','#FF0A00','#FF00FF']))
tp_url = GetTileLayerUrl(tp.visualize(min=0, max=1, palette=['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400','#FF0000']))

In [28]:
map2 = ipyleaflet.Map(
    zoom=2,
    layout={'height':'500px'},
)

map2.add_layer(ipyleaflet.TileLayer(url=t2m_url))
#map2.add_layer(ipyleaflet.TileLayer(url=tp_url))

# Adding the layers control to the map.
map2.add_control(ipyleaflet.LayersControl())
control = FullScreenControl()
map2.add_control(control)

map2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

<hr>

<br>

<a href="./04_aws_open_data_registry.ipynb"><< 04 - AWS - Open Data Registry</a><span style="float:right;"><a href="./11_covid19_case_study_s5p_anomaly_map.ipynb">11 - Covid-19 case study - Sentinel-5P anomaly map>></a></span><br>
<a href="./00_index.ipynb"><< Index</a>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>